# Joint Undergraduate Researcher Onboarding #4
**Topic:** 1) Pymatgen Diffusion 2) Migration Graph

**Date:** April 4, 2022

**Prepared by:** Jiyoon Kim

# Outline & Relevant Documentation

Onboarding Session Demonstration:
* Understand context of diffusion analysis
* Install Pymatgen Diffusion package
* Analyze migration pathways with Migration Graph and ApproxNEB data

Onboarding Independent Exercises Goals:
* Map ApproxNEB data to Migration Graph object to determine best pathway

For further reading and general reference...
* Pymatgen Diffusion Github: https://github.com/materialsvirtuallab/pymatgen-analysis-diffusion
* Migration Graph code: https://github.com/materialsvirtuallab/pymatgen-analysis-diffusion/blob/master/pymatgen/analysis/diffusion/neb/full_path_mapper.py#L64
* Migration Graph example: https://github.com/materialsvirtuallab/pymatgen-analysis-diffusion/blob/master/examples/neb_path_mapping.ipynb

## 1. Diffusion Analysis Context
* Context
    1. Insertion algorithm
        * Insert working ion into low charge density sites
    2. Pathway analysis
        * Create periodic graph with working ion sites as nodes
        * Populate graph with symmetrically equivalent working ion sites
        * Edges that connect nodes are potential migration pathways
    3. ApproxNEB
        * Run calculations to estimate energy barrier of each "edge" or "hop"
        * Energy barrier is calculated using output energies of end points and interpolated "images" between end points
        * Reduce number of calculations by only running calculations for symmetry unique edges
        * Use supercells to avoid finite size effects
    4. Remap energy barrier data to pathway analysis
        * Map supercell sites back to unit cell sites
        * Populate migration graph edges with energy barrier data
        * Find best migration pathway based on lowest energy barrier data

In [1]:
from IPython.display import Image
Image("UC_SC_transformation.png")

FileNotFoundError: No such file or directory: 'UC_SC_transformation.png'

FileNotFoundError: No such file or directory: 'UC_SC_transformation.png'

<IPython.core.display.Image object>

## 2. Pymatgen Diffusion
* https://github.com/materialsvirtuallab/pymatgen-diffusion
* `conda activate <env>`
* `pip install pymatgen-analysis-diffusion`

## 3. Analyze Migration Pathways with Migration Graph and ApproxNEB data

In [2]:
from pymatgen.core import Structure, Element, Lattice, PeriodicSite
from maggma.stores.advanced_stores import MongograntStore
from pprint import pprint
import numpy as np
from pydash import get

ModuleNotFoundError: No module named 'maggma'

In [3]:
hops_store = MongograntStore('ro:mongodb07-ext.nersc.gov/fw_acr_mv', 'bbp_hops')
hops_store.connect()

aneb_store = MongograntStore('ro:mongodb07-ext.nersc.gov/fw_acr_mv', 'approx_neb')
aneb_store.connect()

NameError: name 'MongograntStore' is not defined

### Get Hops Data

In [4]:
battid = '132194_Mg'
hops_doc = hops_store.query_one({'battid':battid})
aneb_doc = aneb_store.query_one({'batt_id':battid})

In [5]:
wion = hops_doc['working_ion']
base_struct = Structure.from_dict(hops_doc['pathmapper']['base_structure'])

In [8]:
print(base_struct.composition)
print(wion)
print(hops_doc['hop_length_cutoff'], 'Å')
print(len(hops_doc['unique_hops']), 'unique hops')
print(aneb_doc['images'].keys())

In [7]:
# Get working ion sites from ApproxNEB doc
base_lattice = Lattice.from_dict(aneb_doc['cep']['base_struct']['lattice'])
sites_from_aneb = []

for h in aneb_doc['cep']['hop_input'].values():
    for i in ['ipos','epos']:
        site = PeriodicSite(species=wion, coords=h[i], lattice=base_lattice)
        sites_from_aneb.append(site)
        
sites_from_aneb = Structure.from_sites(sites_from_aneb)
sites_from_aneb.merge_sites(mode='delete')

In [ ]:
print(sites_from_aneb)

### Create Migration Graph

In [7]:
from pymatgen.analysis.diffusion.neb.full_path_mapper import MigrationGraph

/Users/jiyoonkim/miniconda3/envs/ug-onboard/lib/python3.8/site-packages/pymatgen/analysis/path_finder.py:36: FutureWarning: This code has been superseded by pymatgen.analysis.neb in the separate add-on packagepymatgen-diffusion. This module here is retained for backwards compatibility. It will be removed from2022.1.1.
  warnings.warn(


In [8]:
# Print all hops in migration graph


### Map End Point Supercells and Corresponding Data to Migration Graph

In [9]:
# Get supercell coordinates of a hop's endpoints
sc_coord_i = np.dot(mg.unique_hops[0]['ipos'], aneb_doc['cep']['P_inv'])
sc_coord_e = np.dot(mg.unique_hops[0]['epos'], aneb_doc['cep']['P_inv'])
print(sc_coord_i, mg.unique_hops[0]['ipos'])
print(sc_coord_e, mg.unique_hops[0]['epos'])

In [ ]:
ep0 = base_struct * aneb_doc['cep']['P']
ep0.insert(0, wion, sc_coord_i)

ep1 = base_struct * aneb_doc['cep']['P']
ep1.insert(0, wion, sc_coord_e)

barrier = 950 # made up data

In [10]:
from pymatgen.analysis.diffusion.utils.edge_data_from_sc import add_edge_data_from_sc

### Get Migration Path

In [11]:
from pymatgen.analysis.diffusion.neb.full_path_mapper import get_hop_site_sequence

In [12]:
# Will get error if cost key is not in every path
for u,v,d in mg.m_graph.graph.edges(data=True):
    if 'barrier' not in d.keys():
        d['barrier'] = 9999
    print(d['hop_label'], d['hop'])

In [13]:
# Get best migration path based on barrier


# Onboarding Independent Exercises

Use the aneb doc and migration graph object to populate the migration graph with the following data:
* Task IDs
* Input working ion sites (in supercell) for distance calculations
* ApproxNEB output energies (output energies of end point task docs)
* CrystalNN Coordination Data

In [14]:
from pymatgen.analysis.local_env import CrystalNN
cnn = CrystalNN()